In [1]:
#Finetuning Qwen/Qwen2.5-3B-Instruct with gardeninig dataset


from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    Trainer
)
from peft import LoraConfig, get_peft_model
import torch


# Step 1: Load the Dataset
dataset = load_dataset("mlfoundations-dev/stackexchange_gardening")
# Subsample for quicker iteration during dev:
dataset["train"] = dataset["train"].select(range(6000))
print(dataset["train"].column_names)  # ['instruction', 'completion', 'conversations']


# Step 2: Load Tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"        #This is the base model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Qwen has no separate PAD; align pad_token to eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Step 3: Format to ChatML
def format_row(example):
    messages = [{"role": "system", "content": "You are a helpful gardening assistant."}]
    messages.append({"role": "user", "content": example["instruction"]})
    messages.append({"role": "assistant", "content": example["completion"]})

    if example.get("conversations"):
        for turn in example["conversations"]:
            role = "user" if turn["from"] == "human" else "assistant"
            messages.append({"role": role, "content": turn["value"]})

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # training samples are complete dialogues
    )
    return {"text": text}

dataset = dataset.map(format_row)


# Step 4: Tokenize (dynamic padding)
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=512
    )

tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=dataset["train"].column_names
)


# Step 5: Prepare for PEFT (LoRA)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # # Qwen uses QKV projection layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)


# Step 6: Training Args
training_args = TrainingArguments(
    output_dir="./qwen-gardening-lora",
    max_steps=1000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=100,
    save_steps=200,
    fp16=True,
    report_to=[]
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


# Step 7: Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)

trainer.train()

#Saving the trained model
trainer.save_model("./qwen3B-gardening6k-lora")
tokenizer.save_pretrained("./qwen3B-gardening6k-lora")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/436 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/118M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/42429 [00:00<?, ? examples/s]

['instruction', 'completion', 'conversations']


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
100,1.277200
200,1.247000
300,1.183600
400,1.163300
500,1.227200
600,1.166700
700,1.198700
800,1.201300
900,1.227600
1000,1.190600


('./qwen3B-gardening6k-lora/tokenizer_config.json',
 './qwen3B-gardening6k-lora/special_tokens_map.json',
 './qwen3B-gardening6k-lora/chat_template.jinja',
 './qwen3B-gardening6k-lora/vocab.json',
 './qwen3B-gardening6k-lora/merges.txt',
 './qwen3B-gardening6k-lora/added_tokens.json',
 './qwen3B-gardening6k-lora/tokenizer.json')

In [2]:
# Merging LoRA adapter model into the base Qwen model

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Paths
BASE_MODEL  = "Qwen/Qwen2.5-3B-Instruct"
ADAPTER_DIR = "./qwen3B-gardening6k-lora"   # previously trained adapter model's directory
MERGED_DIR  = "./qwen3B-gardening6k-merged" # Location where the merged model will be saved

# Sanity check: adapter must match the base model
cfg = PeftConfig.from_pretrained(ADAPTER_DIR)
assert cfg.base_model_name_or_path == BASE_MODEL, (
    f"Adapter was trained on {cfg.base_model_name_or_path}, "
    f"but you're merging into {BASE_MODEL}. "
    "Load the exact base used for training."
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load base
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map=None,
)

# Attach LoRA adapter
lora_model = PeftModel.from_pretrained(base_model, ADAPTER_DIR, is_trainable=False)

# Merge LoRA weights into base and unload PEFT
merged_model = lora_model.merge_and_unload()

# Save the merged model & tokenizer
merged_model.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print("Merged model successfully saved to:", MERGED_DIR)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Merged model successfully saved to: ./qwen3B-gardening6k-merged


In [4]:
#Uploading fine-tuned model to huggingface
!pip install huggingface_hub

from huggingface_hub import notebook_login
notebook_login()  # Paste your HF token when prompted


# Upload model to Huggingface
from huggingface_hub import upload_folder

MERGED_DIR = "./qwen3B-gardening6k-merged"  # Path of merged model
REPO_ID = "sanjana040/qwen3B-gardening-finetuned"  # Replace with your HF repo name

upload_folder(
    repo_id=REPO_ID,
    folder_path=MERGED_DIR,
    commit_message="Upload merged Qwen gardening model",
    ignore_patterns=["*.ipynb", "__pycache__/*"]  # optional
)

print(f" Model has been uploaded to: https://huggingface.co/{REPO_ID}")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   4%|4         | 50.3MB / 1.21GB            

  ...g6k-merged/tokenizer.json:   0%|          | 27.6kB / 11.4MB            

  ...0001-of-00002.safetensors:   1%|          | 25.2MB / 4.96GB            

 Model has been uploaded to: https://huggingface.co/sanjana040/qwen3B-gardening-finetuned
